**#Task 4.8**

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os

In [2]:
#defining the path
path = r'C:\Users\jonsa\Documents\Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merged_2.pkl'))

In [4]:
#Identifying column names
ords_prods_merge.columns

Index(['Unnamed: 0.1', 'Unnamed: 0_x', 'order_id', 'user_id', 'order_number',
       'orders_day_of_week', 'order_hour_of_day', 'days_since_previous_order',
       'only_order', 'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'Unnamed: 0_y', 'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day'],
      dtype='object')

In [5]:
#Dropping unnamed columns to remove clutter
ords_prods_merge = ords_prods_merge.drop(['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [6]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_previous_order', 'only_order',
       'product_id', 'add_to_cart_order', 'reordered', '_merge',
       'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day'],
      dtype='object')

In [7]:
#finding the mean, min, and max amounts of orders by department
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

#The table produced here is different from the subset due to the number of rows being analyzed.
#For example, in the subset department 17 had a mean orders of 11.29 however in the full dataframe, it has an average of 15.69
#This is due to the larger volume of data and changes all of the other means listed in the subset as well

In [8]:
#Adding a max_order column that corresponds to user ID
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

#Creating flags for each customer based on spending habits

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] >10), 'loyalty_flag'] = 'Regular customer'

In [11]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <=10, 'loyalty_flag'] = 'New customer'

In [12]:
#printing the table to ensure our code worked correctly
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [13]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

#Interestingly, the newer customers tend to purchase the more expensive products while the loyal customer's tend to buy
the cheaper products

In [14]:
#Creating a column named spending to track individual user stats
ords_prods_merge['spending'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
#Checking table
ords_prods_merge.head(15)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_previous_order,only_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77,7,9.0,Mid-range product,Regular busy day,Regularly busy,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,False,196,1,1,...,77,7,9.0,Mid-range product,Regular busy day,Slowest days,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,False,196,1,1,...,77,7,9.0,Mid-range product,Regular busy day,Slowest days,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,False,196,1,1,...,77,7,9.0,Mid-range product,Least busy day,Slowest days,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,False,196,1,1,...,77,7,9.0,Mid-range product,Least busy day,Slowest days,Most orders,10,New customer,6.367797
5,3367565,1,6,2,7,19.0,False,196,1,1,...,77,7,9.0,Mid-range product,Regular busy day,Regularly busy,Average orders,10,New customer,6.367797
6,550135,1,7,1,9,20.0,False,196,1,1,...,77,7,9.0,Mid-range product,Regular busy day,Busiest days,Average orders,10,New customer,6.367797
7,3108588,1,8,1,14,14.0,False,196,2,1,...,77,7,9.0,Mid-range product,Regular busy day,Busiest days,Most orders,10,New customer,6.367797
8,2295261,1,9,1,16,0.0,False,196,4,1,...,77,7,9.0,Mid-range product,Regular busy day,Busiest days,Most orders,10,New customer,6.367797
9,2550362,1,10,4,8,30.0,False,196,1,1,...,77,7,9.0,Mid-range product,Least busy day,Slowest days,Average orders,10,New customer,6.367797


#Creating flags for the different amounts spent

In [16]:
ords_prods_merge.loc[ords_prods_merge['spending'] < 10, 'spending_flag'] = 'Low spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['spending'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
#Checking values of how many customers are categorized as low spenders versus high spenders
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

#Creating a flag for customers based on the median order frequency

In [19]:
ords_prods_merge['order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_previous_order'].transform(np.median)

In [20]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [21]:
ords_prods_merge.loc[(ords_prods_merge['order_frequency'] > 10) & (ords_prods_merge['order_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [22]:
ords_prods_merge.loc[ords_prods_merge['order_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [23]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

#We can see that there are 21,559,853 frequent customers, 7,208,564 regular customers, and 3,636,437 non-frequent customers.
#These are good results for our business as it means that customers are much more likely to return to our company rather than use a single time

In [24]:
ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge_flags.pkl'))